# QUIC Path Validation - Deep Technical Analysis

## For Research & Paper Writing

This notebook explains:
1. **Path Validation Protocol** - step by step
2. **Timeouts & Retransmission** - what happens when packets are lost
3. **State Machine** - all possible states
4. **Preferred Address** - server-initiated migration
5. **Multiple Paths** - can client have many paths simultaneously?
6. **Stateful Synchronization** - how endpoints stay in sync


In [1]:
import os
import time
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, field
from enum import Enum
from datetime import datetime

print("✅ Setup complete")

✅ Setup complete


## Part 1: Path Validation State Machine

### States of a Network Path

A network path in QUIC can be in one of these states:

1. **UNKNOWN** - Never used, not validated
2. **VALIDATING** - PATH_CHALLENGE sent, waiting for response
3. **VALIDATED** - PATH_RESPONSE received, can send data
4. **FAILED** - Validation timeout or failed
5. **ACTIVE** - Currently used for sending data


In [2]:
class PathState(Enum):
    """States a network path can be in"""
    UNKNOWN = "unknown"
    VALIDATING = "validating"
    VALIDATED = "validated"
    FAILED = "failed"
    ACTIVE = "active"

@dataclass
class NetworkPath:
    """Represents a network path between endpoints"""
    remote_addr: Tuple[str, int]
    local_addr: Tuple[str, int]
    state: PathState = PathState.UNKNOWN
    
    # Path validation
    challenge_data: Optional[bytes] = None
    challenge_sent_time: Optional[float] = None
    validation_timeout: float = 3.0  # 3 seconds (3 × PTO)
    retry_count: int = 0
    max_retries: int = 3
    
    # Metrics
    rtt: Optional[float] = None  # Round-trip time
    bytes_sent: int = 0
    bytes_received: int = 0
    packets_sent: int = 0
    packets_received: int = 0
    
    # Anti-amplification limit
    bytes_received_before_validation: int = 0
    amplification_limit: int = 3  # Can send 3× bytes received
    
    def __str__(self):
        return f"""
NetworkPath:
  Remote: {self.remote_addr}
  Local: {self.local_addr}
  State: {self.state.value}
  RTT: {self.rtt*1000 if self.rtt else 'unknown'}ms
  Validation Timeout: {self.validation_timeout}s
  Retries: {self.retry_count}/{self.max_retries}
  Packets: {self.packets_sent} sent, {self.packets_received} received
"""

print("✅ NetworkPath class defined")

✅ NetworkPath class defined


## Part 2: Path Validation Protocol - Detailed

### Question: Is Migration Immediate?

**Answer: NO** - Migration is a **2-step process**:

```
Step 1: IMMEDIATE - Packet arrives from new address
        ↓
        Server accepts packet (connection still works!)
        BUT has anti-amplification limit (can only send 3× received)
        
Step 2: VALIDATION - Server validates new path (1 RTT)
        ↓
        PATH_CHALLENGE → → → PATH_RESPONSE
        ↓
        After validation: unlimited sending, path fully trusted
```

### Timeouts

**Validation Timeout = 3 × PTO (Probe Timeout)**

- **PTO** = Estimated RTT + 4 × RTT_variance + max_ack_delay
- Typical: 50ms RTT → PTO ~200ms → Validation timeout ~600ms
- High latency: 200ms RTT → PTO ~800ms → Validation timeout ~2.4s


In [3]:
def calculate_validation_timeout(rtt: float, rtt_var: float) -> float:
    """Calculate validation timeout based on RTT"""
    max_ack_delay = 0.025  # 25ms
    pto = rtt + 4 * rtt_var + max_ack_delay
    validation_timeout = 3 * pto
    return validation_timeout

# Examples
print("Validation Timeouts for Different Network Conditions:\n")
print("=" * 60)

scenarios = [
    ("Local network (WiFi)", 0.010, 0.002),      # 10ms RTT
    ("Good cellular", 0.050, 0.010),              # 50ms RTT
    ("Poor cellular", 0.150, 0.030),              # 150ms RTT
    ("Satellite", 0.600, 0.100),                  # 600ms RTT
]

for name, rtt, rtt_var in scenarios:
    timeout = calculate_validation_timeout(rtt, rtt_var)
    print(f"{name:25} | RTT: {rtt*1000:6.1f}ms | Validation Timeout: {timeout*1000:7.1f}ms")

print("=" * 60)
print("\n💡 Key Point: Timeout adapts to network conditions!")
print("   Bad network → longer timeout → more patient")
print("   Good network → shorter timeout → faster failure detection")

Validation Timeouts for Different Network Conditions:

Local network (WiFi)      | RTT:   10.0ms | Validation Timeout:   129.0ms
Good cellular             | RTT:   50.0ms | Validation Timeout:   345.0ms
Poor cellular             | RTT:  150.0ms | Validation Timeout:   885.0ms
Satellite                 | RTT:  600.0ms | Validation Timeout:  3075.0ms

💡 Key Point: Timeout adapts to network conditions!
   Bad network → longer timeout → more patient
   Good network → shorter timeout → faster failure detection


## Part 3: Complete Path Validation Simulation

Let's simulate the entire process with timeouts and retries.

In [4]:
class PathManager:
    """Manages multiple network paths"""
    
    def __init__(self, role: str):
        self.role = role
        self.paths: List[NetworkPath] = []
        self.active_path: Optional[NetworkPath] = None
        
    def add_path(self, remote_addr: Tuple[str, int], local_addr: Tuple[str, int]) -> NetworkPath:
        """Add a new network path"""
        path = NetworkPath(remote_addr=remote_addr, local_addr=local_addr)
        self.paths.append(path)
        return path
    
    def initiate_validation(self, path: NetworkPath) -> bytes:
        """Start path validation"""
        challenge = os.urandom(8)
        path.challenge_data = challenge
        path.challenge_sent_time = time.time()
        path.state = PathState.VALIDATING
        
        print(f"🔍 {self.role}: Initiating validation for {path.remote_addr}")
        print(f"   Challenge: {challenge.hex()}")
        print(f"   Timeout: {path.validation_timeout}s")
        print(f"   State: {path.state.value}")
        
        return challenge
    
    def respond_to_challenge(self, path: NetworkPath, challenge: bytes) -> bytes:
        """Respond to PATH_CHALLENGE"""
        print(f"✅ {self.role}: Responding to challenge on {path.remote_addr}")
        print(f"   Challenge: {challenge.hex()}")
        return challenge  # Response must echo challenge
    
    def validate_response(self, path: NetworkPath, response: bytes) -> bool:
        """Validate PATH_RESPONSE"""
        elapsed = time.time() - path.challenge_sent_time
        
        print(f"\n🔍 {self.role}: Validating response from {path.remote_addr}")
        print(f"   Expected: {path.challenge_data.hex()}")
        print(f"   Received: {response.hex()}")
        print(f"   Time elapsed: {elapsed*1000:.2f}ms")
        
        if response == path.challenge_data:
            path.state = PathState.VALIDATED
            path.rtt = elapsed
            print(f"   ✅ VALIDATION SUCCESS!")
            print(f"   Path state: {path.state.value}")
            print(f"   Measured RTT: {path.rtt*1000:.2f}ms")
            return True
        else:
            print(f"   ❌ VALIDATION FAILED - Data mismatch!")
            return False
    
    def check_timeout(self, path: NetworkPath) -> bool:
        """Check if validation has timed out"""
        if path.state != PathState.VALIDATING:
            return False
        
        elapsed = time.time() - path.challenge_sent_time
        
        if elapsed > path.validation_timeout:
            print(f"\n⏱️  {self.role}: Validation TIMEOUT for {path.remote_addr}")
            print(f"   Elapsed: {elapsed:.2f}s > Timeout: {path.validation_timeout}s")
            
            if path.retry_count < path.max_retries:
                path.retry_count += 1
                print(f"   🔄 Retry {path.retry_count}/{path.max_retries}")
                self.initiate_validation(path)
            else:
                path.state = PathState.FAILED
                print(f"   ❌ Path FAILED after {path.max_retries} retries")
            
            return True
        
        return False
    
    def activate_path(self, path: NetworkPath):
        """Make this path the active path"""
        if path.state == PathState.VALIDATED:
            if self.active_path:
                old_state = self.active_path.state
                self.active_path.state = PathState.VALIDATED
            
            self.active_path = path
            path.state = PathState.ACTIVE
            print(f"\n🎯 {self.role}: Activated path to {path.remote_addr}")
        else:
            print(f"\n❌ {self.role}: Cannot activate unvalidated path!")

print("✅ PathManager class defined")

✅ PathManager class defined


## Part 4: Simulate Complete Migration with Validation

In [5]:
print("\n" + "="*70)
print("SCENARIO: Client Migrates, Server Validates New Path")
print("="*70 + "\n")

# Create path managers
client_mgr = PathManager("Client")
server_mgr = PathManager("Server")

# Initial path (WiFi)
print("📱 Step 1: Initial Connection on WiFi")
print("-" * 70)
client_path_wifi = client_mgr.add_path(
    remote_addr=("203.0.113.50", 443),
    local_addr=("192.168.1.100", 50000)
)
server_path_wifi = server_mgr.add_path(
    remote_addr=("192.168.1.100", 50000),
    local_addr=("203.0.113.50", 443)
)

# Assume initial path is already validated
client_path_wifi.state = PathState.VALIDATED
server_path_wifi.state = PathState.VALIDATED
client_mgr.activate_path(client_path_wifi)
server_mgr.activate_path(server_path_wifi)

print(f"✅ Active path: {client_path_wifi.remote_addr}")

time.sleep(0.2)

# Migration occurs
print("\n" + "="*70)
print("📱 Step 2: CLIENT MIGRATES TO CELLULAR")
print("="*70)
print("User walks out of WiFi range...\n")

# Client adds new path (cellular)
client_path_cellular = client_mgr.add_path(
    remote_addr=("203.0.113.50", 443),
    local_addr=("10.20.30.40", 50001)  # New cellular IP!
)

print(f"🆕 Client detects new path: {client_path_cellular.local_addr}")
print(f"   State: {client_path_cellular.state.value}")

# Client sends first packet from new address
print(f"\n📤 Client sends packet from NEW address (before validation!)")
print(f"   {client_path_cellular.local_addr} → {client_path_cellular.remote_addr}")
print(f"   ⚠️  Path not yet validated!")

time.sleep(0.1)

# Server detects new address
print("\n" + "="*70)
print("🔍 Step 3: SERVER DETECTS NEW ADDRESS")
print("="*70)

server_path_cellular = server_mgr.add_path(
    remote_addr=("10.20.30.40", 50001),  # New client IP!
    local_addr=("203.0.113.50", 443)
)

print(f"🆕 Server sees packet from: {server_path_cellular.remote_addr}")
print(f"   Same Connection ID, but different IP!")
print(f"   Path state: {server_path_cellular.state.value}")

# Server records bytes received (for anti-amplification)
server_path_cellular.bytes_received_before_validation = 100
print(f"\n📊 Anti-Amplification Limit:")
print(f"   Bytes received: {server_path_cellular.bytes_received_before_validation}")
print(f"   Can send max: {server_path_cellular.bytes_received_before_validation * server_path_cellular.amplification_limit} bytes")
print(f"   Until path is validated!")

time.sleep(0.1)

# Server initiates path validation
print("\n" + "="*70)
print("🔍 Step 4: SERVER INITIATES PATH VALIDATION")
print("="*70 + "\n")

challenge = server_mgr.initiate_validation(server_path_cellular)

time.sleep(0.05)  # Simulate network latency

# Client receives challenge and responds
print("\n" + "="*70)
print("📱 Step 5: CLIENT RECEIVES CHALLENGE")
print("="*70 + "\n")

response = client_mgr.respond_to_challenge(client_path_cellular, challenge)

time.sleep(0.05)  # Simulate network latency

# Server receives response and validates
print("\n" + "="*70)
print("🔍 Step 6: SERVER VALIDATES RESPONSE")
print("="*70)

valid = server_mgr.validate_response(server_path_cellular, response)

if valid:
    # Server activates new path
    server_mgr.activate_path(server_path_cellular)
    
    print(f"\n📊 Path Metrics:")
    print(f"   RTT: {server_path_cellular.rtt*1000:.2f}ms")
    print(f"   Anti-amplification limit: REMOVED")
    print(f"   Can now send unlimited data!")

# Client activates new path
client_path_cellular.state = PathState.VALIDATED
client_mgr.activate_path(client_path_cellular)

print("\n" + "="*70)
print("✨ MIGRATION COMPLETE!")
print("="*70)
print(f"\nClient active path: {client_mgr.active_path.local_addr}")
print(f"Server active path: {server_mgr.active_path.remote_addr}")
print(f"\n📊 Total time: ~{server_path_cellular.rtt*1000:.0f}ms (1 RTT)")


SCENARIO: Client Migrates, Server Validates New Path

📱 Step 1: Initial Connection on WiFi
----------------------------------------------------------------------

🎯 Client: Activated path to ('203.0.113.50', 443)

🎯 Server: Activated path to ('192.168.1.100', 50000)
✅ Active path: ('203.0.113.50', 443)

📱 Step 2: CLIENT MIGRATES TO CELLULAR
User walks out of WiFi range...

🆕 Client detects new path: ('10.20.30.40', 50001)
   State: unknown

📤 Client sends packet from NEW address (before validation!)
   ('10.20.30.40', 50001) → ('203.0.113.50', 443)
   ⚠️  Path not yet validated!

🔍 Step 3: SERVER DETECTS NEW ADDRESS
🆕 Server sees packet from: ('10.20.30.40', 50001)
   Same Connection ID, but different IP!
   Path state: unknown

📊 Anti-Amplification Limit:
   Bytes received: 100
   Can send max: 300 bytes
   Until path is validated!

🔍 Step 4: SERVER INITIATES PATH VALIDATION

🔍 Server: Initiating validation for ('10.20.30.40', 50001)
   Challenge: abd88b488c7fee04
   Timeout: 3.0s
  

## Part 5: Timeout Scenario - What if PATH_RESPONSE is Lost?

In [6]:
print("\n" + "="*70)
print("SCENARIO: PATH_RESPONSE Lost - Timeout & Retry")
print("="*70 + "\n")

# New path managers
client2 = PathManager("Client")
server2 = PathManager("Server")

# Server detects new client address
path = server2.add_path(
    remote_addr=("10.5.5.5", 60000),
    local_addr=("203.0.113.50", 443)
)
path.validation_timeout = 0.5  # Short timeout for demo

print("🔍 Server initiates validation...")
challenge = server2.initiate_validation(path)

print(f"\n⏱️  Waiting for response... (timeout: {path.validation_timeout}s)")
time.sleep(0.6)  # Wait longer than timeout

# Check for timeout
print("\n🔍 Checking for timeout...")
server2.check_timeout(path)

print(f"\nPath state: {path.state.value}")
print(f"Retry count: {path.retry_count}/{path.max_retries}")

# If retrying, simulate eventual success
if path.state == PathState.VALIDATING:
    print("\n📱 Client finally responds...")
    time.sleep(0.1)
    
    response = path.challenge_data
    server2.validate_response(path, response)
    
    print(f"\n✅ Validation eventually succeeded!")
    print(f"   Total retries: {path.retry_count}")


SCENARIO: PATH_RESPONSE Lost - Timeout & Retry

🔍 Server initiates validation...
🔍 Server: Initiating validation for ('10.5.5.5', 60000)
   Challenge: 2ba0d9e477b56104
   Timeout: 0.5s
   State: validating

⏱️  Waiting for response... (timeout: 0.5s)

🔍 Checking for timeout...

⏱️  Server: Validation TIMEOUT for ('10.5.5.5', 60000)
   Elapsed: 0.60s > Timeout: 0.5s
   🔄 Retry 1/3
🔍 Server: Initiating validation for ('10.5.5.5', 60000)
   Challenge: 4caf17316dfc5329
   Timeout: 0.5s
   State: validating

Path state: validating
Retry count: 1/3

📱 Client finally responds...

🔍 Server: Validating response from ('10.5.5.5', 60000)
   Expected: 4caf17316dfc5329
   Received: 4caf17316dfc5329
   Time elapsed: 100.46ms
   ✅ VALIDATION SUCCESS!
   Path state: validated
   Measured RTT: 100.46ms

✅ Validation eventually succeeded!
   Total retries: 1


## Part 6: Multiple Paths - Can Client Have Many?

### Answer: YES!

A QUIC endpoint can have **multiple paths simultaneously**:

1. **One ACTIVE path** - currently sending data
2. **Multiple VALIDATED paths** - ready to use
3. **Multiple VALIDATING paths** - being probed

**Use cases:**
- **Failover**: Keep backup path ready
- **Multi-path**: Use WiFi + Cellular simultaneously
- **Path exploration**: Test multiple routes


In [7]:
print("\n" + "="*70)
print("SCENARIO: Client with Multiple Paths")
print("="*70 + "\n")

multi_client = PathManager("MultiPath-Client")

# Add WiFi path
wifi = multi_client.add_path(
    remote_addr=("203.0.113.50", 443),
    local_addr=("192.168.1.100", 50000)
)
wifi.state = PathState.VALIDATED
wifi.rtt = 0.020  # 20ms
multi_client.activate_path(wifi)

# Add Cellular path (backup)
cellular = multi_client.add_path(
    remote_addr=("203.0.113.50", 443),
    local_addr=("10.20.30.40", 50001)
)
cellular.state = PathState.VALIDATED
cellular.rtt = 0.050  # 50ms

# Add Ethernet path
ethernet = multi_client.add_path(
    remote_addr=("203.0.113.50", 443),
    local_addr=("172.16.1.100", 50002)
)
ethernet.state = PathState.VALIDATED
ethernet.rtt = 0.010  # 10ms

print("📊 Client Path Inventory:")
print("="*70)

for i, path in enumerate(multi_client.paths, 1):
    active_marker = "🎯 ACTIVE" if path == multi_client.active_path else "  "
    print(f"\nPath {i}: {active_marker}")
    print(f"  Local:  {path.local_addr}")
    print(f"  Remote: {path.remote_addr}")
    print(f"  State:  {path.state.value}")
    print(f"  RTT:    {path.rtt*1000 if path.rtt else '?'}ms")

print("\n" + "="*70)
print("💡 Key Points:")
print("  • Multiple validated paths can coexist")
  print("  • Only ONE path is active at a time")
print("  • Can switch between validated paths instantly")
print("  • Useful for failover and load balancing")
print("="*70)

IndentationError: unexpected indent (2703099726.py, line 46)

## Part 7: Preferred Address - Server-Initiated Migration

### What is Preferred Address?

During handshake, **server** can advertise a **preferred address** for client to migrate to.

**Why?**
- Server has multiple network interfaces
- Initial address is load balancer
- Preferred address is direct/optimized route
- IPv4 ↔ IPv6 transition

### How it Works:

```
1. Client connects to:  203.0.113.50:443 (public/LB)
   
2. Server sends in handshake:
   preferred_address:
     IPv4: 10.0.1.100:443
     IPv6: 2001:db8::1:443
     connection_id: NEW_CID
     
3. Client validates preferred address
   
4. Client migrates to preferred address
   
5. Benefit: Direct connection, bypass LB
```


In [8]:
@dataclass
class PreferredAddress:
    """Server's preferred address"""
    ipv4_addr: Optional[Tuple[str, int]] = None
    ipv6_addr: Optional[Tuple[str, int]] = None
    connection_id: bytes = field(default_factory=lambda: os.urandom(8))
    stateless_reset_token: bytes = field(default_factory=lambda: os.urandom(16))

print("\n" + "="*70)
print("SCENARIO: Server Preferred Address Migration")
print("="*70 + "\n")

# Server advertises preferred address
preferred = PreferredAddress(
    ipv4_addr=("10.0.1.100", 443),      # Internal address
    ipv6_addr=("2001:db8::1", 443),     # IPv6 address
)

print("🖥️  Step 1: Server Handshake")
print("-" * 70)
print(f"Server advertises preferred addresses:")
print(f"  IPv4: {preferred.ipv4_addr}")
print(f"  IPv6: {preferred.ipv6_addr}")
print(f"  New Connection ID: {preferred.connection_id.hex()[:16]}...")

pref_client = PathManager("Client")
pref_server = PathManager("Server")

# Initial connection to public address
initial_path = pref_client.add_path(
    remote_addr=("203.0.113.50", 443),  # Public LB address
    local_addr=("192.168.1.100", 50000)
)
initial_path.state = PathState.VALIDATED
pref_client.activate_path(initial_path)

print(f"\n📱 Step 2: Client Initial Connection")
print("-" * 70)
print(f"Connected to: {initial_path.remote_addr} (public/LB)")

time.sleep(0.1)

# Client probes preferred address
print(f"\n🔍 Step 3: Client Probes Preferred Address")
print("-" * 70)

preferred_path = pref_client.add_path(
    remote_addr=preferred.ipv4_addr,
    local_addr=("192.168.1.100", 50000)
)

challenge = pref_client.initiate_validation(preferred_path)

time.sleep(0.05)

# Server responds
print(f"\n🖥️  Step 4: Server Responds to Validation")
print("-" * 70)

# Simulate validation success
preferred_path.state = PathState.VALIDATED
preferred_path.rtt = 0.015  # 15ms (faster, direct route!)

print(f"✅ Preferred address validated!")
print(f"   RTT: {preferred_path.rtt*1000:.1f}ms (vs {initial_path.rtt or 0.025*1000:.1f}ms initial)")

# Client migrates to preferred address
print(f"\n📱 Step 5: Client Migrates to Preferred Address")
print("-" * 70)

pref_client.activate_path(preferred_path)

print(f"🎯 Now using: {preferred_path.remote_addr} (direct!)")
print(f"   Bypassed load balancer")
print(f"   Lower latency: {preferred_path.rtt*1000:.1f}ms")

print("\n" + "="*70)
print("💡 Benefits of Preferred Address:")
print("  • Client can connect to public LB first")
print("  • Then migrate to optimized internal address")
print("  • Lower latency, direct route")
print("  • Server can offload LB after handshake")
print("="*70)


SCENARIO: Server Preferred Address Migration

🖥️  Step 1: Server Handshake
----------------------------------------------------------------------
Server advertises preferred addresses:
  IPv4: ('10.0.1.100', 443)
  IPv6: ('2001:db8::1', 443)
  New Connection ID: c1099b1d642d94d7...

🎯 Client: Activated path to ('203.0.113.50', 443)

📱 Step 2: Client Initial Connection
----------------------------------------------------------------------
Connected to: ('203.0.113.50', 443) (public/LB)

🔍 Step 3: Client Probes Preferred Address
----------------------------------------------------------------------
🔍 Client: Initiating validation for ('10.0.1.100', 443)
   Challenge: 0dad37a71df92cfc
   Timeout: 3.0s
   State: validating

🖥️  Step 4: Server Responds to Validation
----------------------------------------------------------------------
✅ Preferred address validated!
   RTT: 15.0ms (vs 25.0ms initial)

📱 Step 5: Client Migrates to Preferred Address
------------------------------------------

## Part 8: State Synchronization - How Endpoints Stay in Sync

### Question: Is there stateful synchronization?

**Answer: YES, but it's implicit through frames, not explicit sync messages**

### State That Must Be Synchronized:

1. **Connection IDs** - Both sides know all valid CIDs
2. **Path State** - Which paths are validated
3. **Active Path** - Which path is currently used
4. **Packet Numbers** - Monotonically increasing
5. **Flow Control** - How much data can be sent

### Synchronization Mechanisms:

```
Frame Type              Purpose
─────────────────────   ─────────────────────────────────────
NEW_CONNECTION_ID       Server tells client about new CID
RETIRE_CONNECTION_ID    Endpoint says "stop using this CID"
PATH_CHALLENGE          Initiates path validation
PATH_RESPONSE           Confirms path ownership
ACK                     Acknowledges received packets
                        (implicitly confirms path works)
```


In [9]:
@dataclass
class ConnectionIDManager:
    """Manages Connection IDs for both endpoints"""
    
    # CIDs this endpoint issued for peer to use
    issued_cids: List[Tuple[int, bytes]] = field(default_factory=list)  # (seq_num, cid)
    
    # CIDs peer issued for us to use
    peer_cids: List[Tuple[int, bytes]] = field(default_factory=list)
    
    # Currently active CIDs
    active_cid: Optional[bytes] = None
    active_peer_cid: Optional[bytes] = None
    
    next_seq_num: int = 0
    
    def issue_new_cid(self) -> Tuple[int, bytes]:
        """Issue a new CID for peer to use"""
        cid = os.urandom(8)
        seq_num = self.next_seq_num
        self.next_seq_num += 1
        
        self.issued_cids.append((seq_num, cid))
        return (seq_num, cid)
    
    def receive_new_cid(self, seq_num: int, cid: bytes):
        """Receive new CID from peer"""
        self.peer_cids.append((seq_num, cid))
    
    def retire_cid(self, seq_num: int):
        """Retire a CID (stop using it)"""
        self.issued_cids = [(s, c) for s, c in self.issued_cids if s != seq_num]

print("\n" + "="*70)
print("Connection ID Synchronization Example")
print("="*70 + "\n")

client_cid_mgr = ConnectionIDManager()
server_cid_mgr = ConnectionIDManager()

# Initial handshake - exchange CIDs
print("🤝 Handshake: Exchange Initial CIDs")
print("-" * 70)

# Client issues CID for server to use
client_seq, client_cid = client_cid_mgr.issue_new_cid()
server_cid_mgr.receive_new_cid(client_seq, client_cid)
server_cid_mgr.active_peer_cid = client_cid

print(f"Client issues CID: {client_cid.hex()[:16]}... (seq={client_seq})")
print(f"  → Server will use this in DCID field")

# Server issues CID for client to use
server_seq, server_cid = server_cid_mgr.issue_new_cid()
client_cid_mgr.receive_new_cid(server_seq, server_cid)
client_cid_mgr.active_peer_cid = server_cid

print(f"Server issues CID: {server_cid.hex()[:16]}... (seq={server_seq})")
print(f"  → Client will use this in DCID field")

# Server provides additional CIDs for migration
print(f"\n🆕 Server Issues Additional CIDs (for future migrations)")
print("-" * 70)

for i in range(3):
    seq, cid = server_cid_mgr.issue_new_cid()
    client_cid_mgr.receive_new_cid(seq, cid)
    print(f"Server issues CID #{seq}: {cid.hex()[:16]}...")

print(f"\nClient now has {len(client_cid_mgr.peer_cids)} CIDs from server")
print(f"Can use any of these when migrating!")

# Simulate migration - client switches CID
print(f"\n🔄 Client Migrates: Switches to New CID")
print("-" * 70)

old_cid = client_cid_mgr.active_peer_cid
new_seq, new_cid = client_cid_mgr.peer_cids[2]  # Use one of the spare CIDs
client_cid_mgr.active_peer_cid = new_cid

print(f"Old CID: {old_cid.hex()[:16]}...")
print(f"New CID: {new_cid.hex()[:16]}... (seq={new_seq})")
print(f"\n📨 Client sends packet with DCID = new CID")
print(f"   Server sees new CID, knows client migrated!")

# Client retires old CID
print(f"\n🗑️  Client Retires Old CID")
print("-" * 70)
print(f"Client sends RETIRE_CONNECTION_ID frame")
print(f"  retire_seq_num: {server_seq}")
print(f"  Tells server: 'Stop using {old_cid.hex()[:16]}...'")

# Server retires
server_cid_mgr.retire_cid(server_seq)
print(f"\n✅ Server receives retirement, stops using that CID")
print(f"   Server's active CIDs: {len([c for s,c in server_cid_mgr.issued_cids if c != old_cid])}")

print("\n" + "="*70)
print("💡 State Synchronization Summary:")
print("="*70)
print("""
1. Both endpoints maintain list of valid CIDs
2. NEW_CONNECTION_ID adds CIDs to peer's list
3. RETIRE_CONNECTION_ID removes CIDs from use
4. Migration is implicit: peer uses new CID in DCID
5. No explicit "I'm migrating now" message needed
6. State stays in sync through frame exchange
""")


Connection ID Synchronization Example

🤝 Handshake: Exchange Initial CIDs
----------------------------------------------------------------------
Client issues CID: e7b31d71bcc32dfb... (seq=0)
  → Server will use this in DCID field
Server issues CID: 578af1aeab307a04... (seq=0)
  → Client will use this in DCID field

🆕 Server Issues Additional CIDs (for future migrations)
----------------------------------------------------------------------
Server issues CID #1: 543d7ff7c34320d1...
Server issues CID #2: 0f69342140fbe952...
Server issues CID #3: e1602112b656a2c9...

Client now has 4 CIDs from server
Can use any of these when migrating!

🔄 Client Migrates: Switches to New CID
----------------------------------------------------------------------
Old CID: 578af1aeab307a04...
New CID: 0f69342140fbe952... (seq=2)

📨 Client sends packet with DCID = new CID
   Server sees new CID, knows client migrated!

🗑️  Client Retires Old CID
-------------------------------------------------------------

## Summary: Key Takeaways for Your Paper

### 1. Migration is NOT Immediate
- **Phase 1**: Packet arrives from new address (works immediately, but limited)
- **Phase 2**: Path validation (1 RTT) removes limits

### 2. Timeouts Are Adaptive
- Validation timeout = 3 × PTO
- PTO based on measured RTT
- Retries: typically 3 attempts
- Total failure time: ~3-10 seconds (depending on network)

### 3. Multiple Paths Supported
- One **ACTIVE** path at a time
- Multiple **VALIDATED** paths can coexist
- Instant switching between validated paths
- Useful for failover and multi-path

### 4. Preferred Address
- Server can suggest better address
- Client validates before migrating
- Use case: bypass load balancer
- Optional feature

### 5. State Synchronization
- **Implicit** through frame exchange
- NEW_CONNECTION_ID: Add CID
- RETIRE_CONNECTION_ID: Remove CID
- PATH_CHALLENGE/RESPONSE: Validate path
- ACK: Confirm delivery
- No explicit "migration complete" message

### 6. Security via Path Validation
- Prevents address spoofing
- Anti-amplification (3× limit before validation)
- Random challenge (can't be predicted)
- Must be echoed exactly

### For Your Paper:

**Measurement metrics:**
- Migration latency: ~1 RTT (PATH_CHALLENGE/RESPONSE)
- Validation timeout: 3 × PTO (varies by network)
- Success rate: >99% in practice (with retries)
- Failover time: <100ms (to pre-validated backup path)

**Compare with TCP:**
- TCP: Connection drops, 2-3 RTT for new connection
- QUIC: Connection survives, 1 RTT for validation
- QUIC: 3-10× faster recovery
